## Business Problem

A client in King County, WA wants to advise homeowners on **home improvement projects** that will **add to the sale value of their homes**.

**This advice should be based on data from the most recent full calendar year, 2019**.

### Core questions:
Enclosing a porch will increase the sale price of a home.

Converting a garage to a bedroom is a good way to increase the sale price of a home.

Upgrading to a forced-air heating system will increase the sale price of a home.

### Core Goals:
Create model

Interpret results

Make recomendations

### Schedule:
Friday: Business Understanding and Data Importation

Saturday: Data Understanding. Add .gitignore file in exploratory directory.

Sunday: Data Prep

### Importing Libraries

In [157]:
# import modules for eda and plotting
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Pandas Dataframe

In [158]:
# setting the number of rows displayed
pd.options.display.max_rows = 200

### Importing the Data

In [159]:
# create paths to the files
files = ['EXTR_RPSale.csv', 'EXTR_ResBldg.csv', 'EXTR_Parcel.csv', 'EXTR_LookUp.csv']
paths = [f'../../data/raw/{file}' for file in files]

# create list of data frames, importing data as strings
dfs = [pd.read_csv(path, dtype=str) for path in paths]

# isolate individual data frames
df_sale = dfs[0]
df_resb = dfs[1]
df_parc = dfs[2]
df_look = dfs[3]

In [160]:
# checking shape of the dataframes
df_sale.shape, df_resb.shape, df_parc.shape, df_look.shape

((351067, 24), (181510, 50), (205199, 82), (1208, 3))

In [161]:
# checking columns and nulls
df_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351067 entries, 0 to 351066
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ExciseTaxNbr        351067 non-null  object
 1   Major               351067 non-null  object
 2   Minor               351067 non-null  object
 3   DocumentDate        351067 non-null  object
 4   SalePrice           351067 non-null  object
 5   RecordingNbr        351067 non-null  object
 6   Volume              351067 non-null  object
 7   Page                351067 non-null  object
 8   PlatNbr             351067 non-null  object
 9   PlatType            351067 non-null  object
 10  PlatLot             351067 non-null  object
 11  PlatBlock           351067 non-null  object
 12  SellerName          351067 non-null  object
 13  BuyerName           351067 non-null  object
 14  PropertyType        351067 non-null  object
 15  PrincipalUse        351067 non-null  object
 16  Sa

In [162]:
# checking columns and nulls
df_resb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181510 entries, 0 to 181509
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Major               181510 non-null  object
 1   Minor               181510 non-null  object
 2   BldgNbr             181510 non-null  object
 3   NbrLivingUnits      181510 non-null  object
 4   Address             181510 non-null  object
 5   BuildingNumber      181510 non-null  object
 6   Fraction            181510 non-null  object
 7   DirectionPrefix     181146 non-null  object
 8   StreetName          181510 non-null  object
 9   StreetType          181510 non-null  object
 10  DirectionSuffix     181146 non-null  object
 11  ZipCode             154594 non-null  object
 12  Stories             181510 non-null  object
 13  BldgGrade           181510 non-null  object
 14  BldgGradeVar        181510 non-null  object
 15  SqFt1stFloor        181510 non-null  object
 16  Sq

In [163]:
# checking columns and nulls
df_parc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205199 entries, 0 to 205198
Data columns (total 82 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Unnamed: 0              205199 non-null  object
 1   Major                   205199 non-null  object
 2   Minor                   205199 non-null  object
 3   PropName                196088 non-null  object
 4   PlatName                176654 non-null  object
 5   PlatLot                 205199 non-null  object
 6   PlatBlock               205199 non-null  object
 7   Range                   205199 non-null  object
 8   Township                205199 non-null  object
 9   Section                 205199 non-null  object
 10  QuarterSection          205199 non-null  object
 11  PropType                205199 non-null  object
 12  Area                    205193 non-null  object
 13  SubArea                 205193 non-null  object
 14  SpecArea                4864 non-nul

In [164]:
# checking problematic columns
df_prob = df_parc[['Unnamed: 0', 'PropName', 'PlatName', 'Area', 'SubArea', 'SpecArea', 'SpecSubArea' ]]
df_prob.head()

,Unnamed: 0,PropName,PlatName,Area,SubArea,SpecArea,SpecSubArea
0,0,,SUMMER RIDGE DIV NO. 02,35,2,NaN,NaN
1,2,,SANDER'S TO GILMAN PK & SALMON BAY,19,1,NaN,NaN
2,3,,VASHON GARDENS ADD,100,3,NaN,NaN
3,6,,NaN,1,1,NaN,NaN
4,7,,ELDORADO NORTH,37,2,NaN,NaN


In [165]:
df_parc = df_parc.drop('Unnamed: 0', axis=1)

In [166]:
# checking dropped column
df_parc.head(1)

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
0,807841,0410,,SUMMER RIDGE DIV NO. 02,41,,6,25,22,SW,...,N,N,N,N,N,N,N,N,N,N


In [167]:
df_look.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208 entries, 0 to 1207
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LUType         1208 non-null   object
 1   LUItem         1208 non-null   object
 2   LUDescription  1208 non-null   object
dtypes: object(3)
memory usage: 28.4+ KB


In [168]:
df_look.head(1)

,LUType,LUItem,LUDescription
0,1,1,LAND ONLY ...


### Merging the data frames into a master df.

In [169]:
df = pd.merge(pd.merge(df_sale, df_parc, on=['Major', 'Minor']), df_resb, on=['Major', 'Minor'])

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251300 entries, 0 to 251299
Columns: 151 entries, ExciseTaxNbr to AddnlCost
dtypes: object(151)
memory usage: 291.4+ MB


In [171]:
df.head(2)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,2743355,638580,0110,07/14/2015,190000,20150715002686,,,,,...,1,0,1,1963,0,0,0,0,3,0
1,2743356,638580,0110,07/14/2015,0,20150715002687,,,,,...,1,0,1,1963,0,0,0,0,3,0


## SQL Dataframe

### SQL Prelim Work

In [172]:
# importing sqlite
import sqlite3

# creating database, connection, and cursor
conn = sqlite3.connect('KingDB.db')  
cur = conn.cursor()

# creating query fetch function
def fetcha(q):
    """Returns an SQL query."""
    return cur.execute(q).fetchall()

In [173]:
# getting table names
q = """SELECT name FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1"""
fetcha(q)

[('PARC',), ('RESB',), ('SALES',)]

In [174]:
# joining tables to create dataframe
q = """SELECT*FROM SALES AS SA
       JOIN PARC AS PA
       ON SA.Major = PA.Major
       AND SA.Minor = PA.Minor
       JOIN RESB AS RE
       ON PA.Major = RE.Major
       AND PA.Minor = RE.Minor
       """
df2 = pd.DataFrame(fetcha(q))
df2.columns = [i[0] for i in cur.description]

In [175]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251300 entries, 0 to 251299
Columns: 156 entries, ExciseTaxNbr to AddnlCost
dtypes: object(156)
memory usage: 299.1+ MB


In [176]:
df2.shape

(251300, 156)

In [177]:
df2.head(1)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,2743355,638580,0110,07/14/2015,190000,20150715002686,,,,,...,1,0,1,1963,0,0,0,0,3,0


In [178]:
# dropping unnamed column
df2 = df2.drop('Unnamed: 0', axis=1)

In [179]:
df2.shape

(251300, 155)

In [180]:
df2.isna().sum()

ExciseTaxNbr                   0
Major                          0
Minor                          0
DocumentDate                   0
SalePrice                      0
RecordingNbr                   0
Volume                         0
Page                           0
PlatNbr                        0
PlatType                       0
PlatLot                        0
PlatBlock                      0
SellerName                     0
BuyerName                      0
PropertyType                   0
PrincipalUse                   0
SaleInstrument                 0
AFForestLand                   0
AFCurrentUseLand               0
AFNonProfitUse                 0
AFHistoricProperty             0
SaleReason                     0
PropertyClass                  0
SaleWarning                    0
Major                          0
Minor                          0
PropName                   11473
PlatName                   29223
PlatLot                        0
PlatBlock                      0
Range     

## 2019 Data

#### Light Data Cleaning

Changed datestrings to datetime objects, saleprice strings to floats in the main pandas dataframe. Also stripped blank spaces from the lookup types in the lookup dataframe

In [181]:
# changing datestrings to datetime objects
df.DocumentDate = pd.to_datetime(df.DocumentDate)

# adding a document year column
df['DocumentYear'] = df['DocumentDate'].apply(lambda x: x.year)

# converting SalePrice string to float
df['SalePrice'] = df['SalePrice'].astype('float')

# stripping blank spaces from lookup strings
df_look['LUType'] = df_look['LUType'].apply(lambda x: x.strip())

In [182]:
# isolating 2019 data
df19 = df[df['DocumentYear']==2019]

In [183]:
df19.shape

(43838, 152)

In [184]:
df19.isnull().sum()

ExciseTaxNbr                  0
Major                         0
Minor                         0
DocumentDate                  0
SalePrice                     0
RecordingNbr                  0
Volume                        0
Page                          0
PlatNbr                       0
PlatType                      0
PlatLot_x                     0
PlatBlock_x                   0
SellerName                    0
BuyerName                     0
PropertyType                  0
PrincipalUse                  0
SaleInstrument                0
AFForestLand                  0
AFCurrentUseLand              0
AFNonProfitUse                0
AFHistoricProperty            0
SaleReason                    0
PropertyClass                 0
SaleWarning                   0
PropName                   2434
PlatName                   5151
PlatLot_y                     0
PlatBlock_y                   0
Range                         0
Township                      0
Section                       0
QuarterS

In [186]:
# creating a datafrme of 2019 data with non-zero sales prices
non_zero = df19[df19['SalePrice'] !=0]

In [132]:
non_zero.shape

(29944, 152)

In [130]:
# nz = non_zero.T.drop_duplicates().T

In [133]:
nz.shape

(29944, 138)

In [134]:
df.isnull().sum()

ExciseTaxNbr                   0
Major                          0
Minor                          0
DocumentDate                   0
SalePrice                      0
RecordingNbr                   0
Volume                         0
Page                           0
PlatNbr                        0
PlatType                       0
PlatLot_x                      0
PlatBlock_x                    0
SellerName                     0
BuyerName                      0
PropertyType                   0
PrincipalUse                   0
SaleInstrument                 0
AFForestLand                   0
AFCurrentUseLand               0
AFNonProfitUse                 0
AFHistoricProperty             0
SaleReason                     0
PropertyClass                  0
SaleWarning                    0
PropName                   11473
PlatName                   29223
PlatLot_y                      0
PlatBlock_y                    0
Range                          0
Township                       0
Section   

In [148]:
df_look[df_look['LUType']=='108']

,LUType,LUItem,LUDescription
243,108,1,Floor-Wall ...
244,108,2,Gravity ...
245,108,3,Radiant ...
246,108,4,Elec BB ...
247,108,5,Forced Air ...
248,108,6,Hot Water ...
249,108,7,Heat Pump ...
250,108,8,Other ...


In [156]:
df_look[df_look['LUType']=='82']

,LUType,LUItem,LUDescription
1150,82,1,1 Cabin ...
1151,82,10,10 Very Good ...
1152,82,11,11 Excellent ...
1153,82,12,12 Luxury ...
1154,82,13,13 Mansion ...
1155,82,2,2 Substandard ...
1156,82,20,Exceptional Properties ...
1157,82,3,3 Poor ...
1158,82,4,4 Low ...
1159,82,5,5 Fair ...


In [150]:
df_look

,LUType,LUItem,LUDescription
0,1,1,LAND ONLY ...
1,1,10,Land with new building ...
2,1,11,"Household, single family units ..."
3,1,12,"Multiple family residence (Residential, 2-4 un..."
4,1,13,"Multiple family residence (Residential, 5+ uni..."
...,...,...,...
1203,99,1,SUBSTANDARD ...
1204,99,2,BELOW AVERAGE ...
1205,99,3,AVERAGE ...
1206,99,4,ABOVE AVERAGE ...


In [194]:
def lookup(code):
    """Returns a data frame with rows containing the specified lookup code."""
    return df_look[df_look['LUType']==f'{code}']

In [195]:
lookup(103)

,LUType,LUItem,LUDescription
213,103,1,SINGLE FAMILY ...
214,103,10,AMUSEMENT/ENTERTAINMENT ...
215,103,11,CULTURAL ...
216,103,12,EDUCATIONAL SERVICE ...
217,103,13,COMMERCIAL SERVICE ...
218,103,14,RETAIL/WHOLESALE ...
219,103,15,MANUFACTURING ...
220,103,16,AGRICULTURAL ...
221,103,17,FORESTRY ...
222,103,18,FISH & WILDLIFE MGMT ...
